# Module: Binning of Metagenomic Assembly

Binning is a critical step in metagenomic analysis that involves grouping assembled contigs into clusters, each representing a distinct organism or genome. This process enables researchers to reconstruct individual genomes from complex microbial communities, facilitating the study of their functional roles, ecological interactions, and evolutionary relationships.  Binning is especially valuable for studying uncultivated microorganisms, providing insights into the "microbial dark matter" and expanding our understanding of biodiversity in diverse environments.

In this notebook, we will explore how to group assembled metagenomic reads into bins, and subsequently, assess the quality of these bins to determine if they would qualify as a metagenome-assembled genome (MAG).

Created by: _Microbial Oceanography Laboratory (MOLab)_

---
## How to Use This Notebook

1. Make sure tools are installed already (see below if not yet).
2. Activate environment. Replace environment name accordingly.
```bash
conda activate binning-env
```
2. Open jupyter notebook with the command below and select the notebook.
```bash
jupyter notebook
```
3. To run the cells in this notebook, press Shift+Enter.

---
## Tools Used
1. **MetaBAT2**
2. **CheckV**
3. **CheckM**
4. **EukCC**

To install these tools, find the `binning.yaml` file located in the same folder as this notebook (in repository). Then run the command below in the terminal:

```bash
conda env create -f binning.yaml
```

---
## Starting Files 

1. Contigs FASTA file (see **Assembly Module**).
2. BAM mapping files (see **Mapping Module**).

---
## Expected Outputs

1. FASTA file of MAGs.
2. Assessment reports of bins.

---
## Table of Contents
 * [**Binning**](#Binning)
    * [Calculate Contig Coverage](#Calculate-Contig-Coverage)
    * [Bin Contigs](#Bin-Contigs)
 * [**Assessment of Bins**](#Assessment-of-Bins)
    * [CheckM v1](#CheckM-v1)
    * [EukCC](#EukCC)
        * [Setup EukCC Database](#Setup-EukCC-Database)
        * [Run EukCC](#Run-EukCC)
    * [CheckV](#CheckV)
        * [Setup CheckV Database](#Setup-CheckV-Database)
        * [Concatenate Contigs](#Concatenate-Contigs)
        * [Run CheckV](#Run-CheckV)
        * [Inspect CheckV Output](#Inspect-CheckV-Output)
    * [Other](#Other)

----
# <font color = 'gray'>Binning</font>

Shotgun metagenomics generates a pool of DNA sequences from all organisms within a given community, making genome recovery a complex task. However, distinctive genomic properties—such as nucleotide composition and abundance—can help differentiate one genome from another. Leveraging these properties, various bioinformatics tools have been developed to group metagenomic contigs into bins that ideally represent individual genomes.

In this section, we will use `metabat2` for binning. Briefly, `metabat2` clusters contigs by analyzing tetranucleotide frequency (TNF; the frequency of 4-mers) and contig abundance patterns.

### Calculate Contig Coverage

`metabat2` works best if we have information about the coverages of the contigs. This can be calculated from the mapping of reads to the assembly (see **Mapping Module**). Fortunately, `metabat2` provides a script that performs contig depth calculation. The command below requires BAM alignment files to be present in the working directory. This will then produce a file describing the depth of each contig in the assembly.

In [ ]:
!jgi_summarize_bam_contig_depths \
    –outputDepth summary_contig_depth.txt \
    *.bam

### Bin Contigs

With the contig coverage information calculated, `metabat2` can now be executed as demonstrated below.

The arguments used in the command are:

| option/input | description |
| :-: | :- |
| `-i` | Contigs generated from assembly (see **Assembly Module**). |
| `-a` | Summary of contig coverage (see **Mapping Module**). |
| `-o` | Base filename for each bin. |
| `-m` | Minimum contig size (default = 2,500 bp). |

In [ ]:
!metabat2 \
    -i contigs.fa \
    -a summary_contig_depth.txt \
    -o metabat2_out/bin \
    -m 1500

The command will generate one FASTA file for each bin inside the output folder (`metabat2_out`).

----
# <font color = 'gray'>Assessment of Bins</font>

For prokaryotes and eukaryotes, the quality of bins is typically assessed using two key metrics: completeness and contamination. Many tools, including the ones we will use below, estimate these metrics by leveraging a set of single-copy marker genes (SCMGs). As the name suggests, SCMGs are genes expected to appear as a single copy in a genome, though this set can vary across different taxonomic lineages.

Different tools have their nuances on how they estimate completeness and contamination but the general idea is as follows: To calculate completeness, we start with a reference set of _m_ SCMGs and identify how many of these are present in each bin. Completeness is expressed as the proportion of unique SCMGs found in a bin relative to the total number of reference SCMGs (_m_). Contamination, on the other hand, measures the presence of multiple copies of an SCMG within a bin. The occurrence of duplicate SCMGs may suggest that contigs from other genomes have been incorrectly grouped with the bin. Both completeness and contamination are expressed as percentages, and arbitrary thresholds are often used to categorize bins by quality.

While useful, this type of assessment has limitations. Relying solely on SCMGs does not provide a holistic view of an organism's genomic landscape. Additionally, at low completeness levels, contaminating SCMGs may go undetected, artificially boosting completeness and underestimating contamination. Despite these caveats, completeness and contamination serve as reasonable proxies for bin quality, particularly when prioritizing high-quality bins for downstream analysis.

You may find the following reading interesting as well: [Why genome completeness and contamination estimates are more complicated than you think](https://microbe.net/2017/12/13/why-genome-completeness-and-contamination-estimates-are-more-complicated-than-you-think/).

Below we will use different tools to assess the quality of the inferred bins.

### CheckM v1

CheckM provides a set of tools for assessing the quality of genomes recovered from isolates, single cells, or metagenomes. It provides robust estimates of genome completeness and contamination by using collocated sets of genes that are ubiquitous and single-copy within a phylogenetic lineage. Assessment of genome quality can also be examined using plots depicting key genomic characteristics (e.g., GC, coding density) which highlight sequences outside the expected distributions of a typical genome. CheckM also provides tools for identifying genome bins that are likely candidates for merging based on marker set compatibility, similarity in genomic characteristics, and proximity within a reference genome tree.

_Source: [CheckM Wiki](https://github.com/Ecogenomics/CheckM/wiki/Introduction#about)_

Besides the quoted description above, another important consideration is that `checkm` was developed particularly to assess the genomes of **prokaryotic organisms**. For eukaryotes and viruses, see `eukcc` and `checkv`, respectively, below.

<div class="alert alert-block alert-warning">
<b>For Future Users:</b> 
    
CheckM has now a version 2 which uses a machine learning model to estimate completeness and contamination of prokaryotic bins instead of SCMGs. According to the authors, CheckM v2 provides more accurate estimates, although they still recommend a comparison with CheckM v1. For future users of this module, consider incorporating CheckM v2 in your workflow.
</div>

The command below is a wrapper of multiple utilities available in the `checkm` package. It runs the entire `checkm` workflow (gene prediction, placement of bin to the genome tree, identification of lineage-specific marker genes, ...). The arguments specified are:

| Parameter | Description |
| :-: | :- |
| `-x` | File extension of the bin(s). |
| `metabat2_out` | The folder containing the inferred bins. |
| `checkm_out` | Output folder. |

More details about this command can be found [here](https://github.com/Ecogenomics/CheckM/wiki/Workflows#lineage-specific-workflow) or by running the help option (`checkm lineage_wf -h`).

In [ ]:
!checkm lineage_wf \
    -x fa \
    metabat2_out \
    checkm_out

After the command finishes running, the terminal will output a tabular structure showing various metrics for each bin. These metrics are well-documented in this webpage: [CheckM Reported Statistics](https://github.com/Ecogenomics/CheckM/wiki/Reported-Statistics).

Moreover, you can see more detailed reports in the output folder (`checkm_out`). The `checkm_out/bins` folder contains predicted genes of each bin, while the `checkm_out/storage` includes similar metrics shown in the standard output (`checkm_out/storage/bin_stats*`), tree files, and others related to the marker genes used. 

To simplify everything, what we're primarily interested in here are the completeness and contamination estimates. There are no general consensus on what is considered a "good" bin, but for the sake of having well-defined boundaries, we use the guidelines discussed in Table 1 of this [paper](https://pmc.ncbi.nlm.nih.gov/articles/PMC6436528/) (summarized below). We also use this guideline for evaluation of eukaryotic bins using `eukcc`.

| Assembly Quality | Criterion |
| :-: | :- |
| Finished | Single continuous sequence (for prokaryotes) without gaps or ambiguities (Ns). |
| High-quality Draft | Completeness $>$ 90%, Contamination $<$ 5% |
| Medium-quality Draft | Completeness $\geq$ 50%, Contamination $<$ 10% |
| Low-quality Draft | Completeness $<$ 50%, Contamination $<$ 10% |

## EukCC

`eukcc` generally works in the same manner as CheckM, albeit, it is catered for eukaryotic organisms.

### Setup EukCC Database

Before you can run `eukcc` for the first time, you have to initially setup the database. The steps below outline how to accomplish this. First, create a directory, download the database (`eukcc2_db_ver_1.1.tar.gz`), then extract it.

In [ ]:
%%bash

mkdir eukccdb
cd eukccdb
wget http://ftp.ebi.ac.uk/pub/databases/metagenomics/eukcc/eukcc2_db_ver_1.1.tar.gz
tar -xzvf eukcc2_db_ver_1.1.tar.gz

Afterwards, to permanently set the directory to the database as an environment variable, copy the command below to your `.bashrc` file

```
export EUKCC2_DB=/path/to/.../eukcc2_db_ver_1.1
```

Make sure to specify the absolute file path of the `eukcc` database.

### Run EukCC

Now, `eukcc` can be executed by simply running the command below. This will create a report for all bins inside the `metabat_out` folder.

In [ ]:
!eukcc folder \
    --out eukcc_out \
    metabat_out

Inside the output folder (`eukcc_out`), a file named `eukcc.csv` contains the completeness and contamination estimates for your bins. Again, we use the same criteria mentioned in the section about `checkm` to categorize bins according to quality.

## CheckV

Unlike the methods described earlier, `checkv` does not rely on single-copy marker genes (SCMGs) to assess completeness and contamination. This is because viruses lack a universal set of SCMGs. Instead, `checkv` estimates completeness by comparing the length of the bin to the typical genome length of a closely related virus. This approach is based on the observation that viral genomes tend to have consistent lengths, even among viruses with low amino acid identity. However, this method may produce inaccurate estimates for novel or highly divergent viruses.

For contamination, `checkv` identifies and removes host-derived sequences from contigs, effectively cleaning the bin. Unlike completeness, contamination is not reported as a percentage, as the focus is on directly eliminating host sequences rather than quantifying their presence.

### Setup CheckV Database

Before running `checkv`, we have to setup its database first. To download its database, run the following command.

In [ ]:
!checkv download_database ./

Afterwards, similar to `eukcc`'s database, set the path to the database as environment variable by putting the command below in the `.bashrc` configuration file. This `checkv` to easily locate the database everytime it is called.

```
export CHECKVDB=/path/to/checkv-db
```

Make sure to specify whatever path is applicable to your case.

### Concatenate Contigs

In this section, we follow the workflow outlined in [ViroProfiler](https://pmc.ncbi.nlm.nih.gov/articles/PMC10072060/) to execute `checkv`.

Since `checkv` operates on single-contig viral bins or genomes, multi-contig bins must first be concatenated into a single sequence. To achieve this, we use a custom script, `concat_vrhyme_bin.py`, developed by the authors of ViroProfiler. This script concatenates the contigs within each bin by inserting a fixed number of ambiguous bases ("N") between them. The code below processes all FASTA files located in the `metabat_out` folder, applying this concatenation step before running `checkv`.

In [ ]:
!mkdir concat_bins

In [ ]:
import os

for fasta in os.listdir('metabat_out'):
    if fasta.endswith('.fa'):
        !./concat_vrhyme_bin.py \
            -i metabat_out/{fasta} \
            -o concat_bins/{fasta}

### Run CheckV

Next, we use the `checkv` pipeline to assess the quality of the viral bins.

In [ ]:
!mkdir check_bins

In [ ]:
import os

for bin in os.listdir('concat_bins'):
    if bin.endswith(".fasta"):
        print("\n===== CHECKING BIN: {}\n".format(bin))
        
        !mkdir check_bins/{bin}
        !checkv end_to_end \
            concat_bins/{bin} \
            check_bins/{bin} \
            -d /mnt/e/Shotgun Metagenomics Workshop/VM_shared_folder/viral_pipeline/0-other_utils/checkv/checkv-db-v1.5

### Inspect CheckV Output

`checkv`'s completeness estimate can be seen inside the `check_bins/*/completeness.tsv` file. If you only have a few bins, you can easily inspect it manually. But if you have many bins, you can use the code below to extract the quality for each bin.

In [ ]:
for bin in os.listdir('check_bins'):
    print(f'\n\n===== {bin}')
    awk_checkv_quality = f"awk -F '\t' 'NR > 1 {{print $8}}' check_bins/{bin}/quality_summary.tsv"
    !{awk_checkv_quality}

## Other

In addition to the tools mentioned above, you can evaluate your bins by comparing them to a closely related reference genome. To perform this analysis, first assign a taxonomic group to the bin (see **Contig Level Taxonomic Annotation Module**) and then use QUAST for a reference-based assessment (see **Assembly Module**).